In [2]:
import numpy as np
import os
#print(os.sys.path)
import cv2
#!pip install scikit-image
from skimage.feature import local_binary_pattern

['', 'C:\\Ana_conda\\envs\\workshop\\python35.zip', 'C:\\Ana_conda\\envs\\workshop\\DLLs', 'C:\\Ana_conda\\envs\\workshop\\lib', 'C:\\Ana_conda\\envs\\workshop', 'C:\\Ana_conda\\envs\\workshop\\lib\\site-packages', 'C:\\Ana_conda\\envs\\workshop\\lib\\site-packages\\win32', 'C:\\Ana_conda\\envs\\workshop\\lib\\site-packages\\win32\\lib', 'C:\\Ana_conda\\envs\\workshop\\lib\\site-packages\\Pythonwin', 'C:\\Ana_conda\\envs\\workshop\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\HP\\.ipython']
  Stored in directory: C:\Users\HP\AppData\Local\pip\Cache\wheels\68\f8\29\b53346a112a07d30a5a84d53f19aeadaa1a474897c0423af91
  Stored in directory: C:\Users\HP\AppData\Local\pip\Cache\wheels\f4\0c\f6\ce6b2d1aa459ee97cc3c0f82236302bd62d89c86c700219463
Successfully built networkx toolz


In [3]:
import os
directory_list = list()
dir_path = "dataset"
for root, dirs, files in os.walk(dir_path, topdown=False):
    for name in dirs:
        directory_list.append((name))
#print(directory_list)
for directory in directory_list:
    print(directory)

catagory_0
catagory_1
catagory_2


In [4]:
import glob

labels = 'labels.csv'
labelfile = open(labels,'w')
for i in range(len(directory_list)):
    readpath = (dir_path+'/' + directory_list[i]+'/*jpg')
    #print(readpath)
    images = glob.glob(readpath)
    for image in images:
        labelfile.write(image+','+directory_list[i][9:len(directory_list[i])]+'\n')
labelfile.close()

In [4]:
import random
labels = 'labels.csv'
shuffled_labels = 'Shuffled_labels.csv'

labelfile = open(labels, "r")
lines = labelfile.readlines()
labelfile.close()
random.shuffle(lines)

shufflefile = open(shuffled_labels, "w")
shufflefile.writelines(lines)
shufflefile.close()

In [5]:
#to check uniform or not 
def uniform(pattern):
    pat= int(pattern)
    a=0
    b=0
    cnt=0
    for i in range(0,8):
        if( i==0 ):
            a= int(pattern/2**(7-i))
        else:
            b= int(pattern/2**(7-i))
            if(b!=a):
                cnt=cnt+1
                a=b
        pattern=pattern%2**(7-i)
    if(cnt<=2):
        return 1 #uniform hbe
    else:
        return 0
items = []
true_items = []
for i in range(0, 256):
    if uniform(i):
        items.append(i)
        true_items.append(i)
        
items.append(250)
true_items.append(250)
print(len(items))


59


In [6]:
import cv2
import numpy as np
def get(image, idx, idy):
    if idx < (len(image)) and idy < len(image[0]) and idx>=0 and idy >=0:
        return 1
    else:
        return 0
    
def dir(x,y,img):
	ax=int(img[x+1,y])-int(img[x,y])
	ay=int(img[x,y-1])-int(img[x,y])
    
	if ax>=0 and ay>=0:
		return 1
	elif ax<0 and ay>=0:
		return 2
	elif ax<0 and ay<0:
		return 3
	elif ax>=0 and ay<0:
		return 4
		
def d_lbp(x,y,img):
    out=[]
    fx=[0,-1,-1,-1,0,1,1,1]
    fy=[1,1,0,-1,-1,-1,0,1]
    val = img[x][y]
    for i in range(0, 8):
        n_x = x+fx[i]
        n_y = y+fy[i]
        val1=img[n_x][n_y]
        if val1>=val:
            out.append(1)
        else:
            out.append(0)
    #print(out)
    return out

def add_dict(dic, val):
    if val not in dic.keys():
        dic[250] = dic[250] + 1
    else:
        dic[val] = dic[val] + 1

def lbp(img, dic1):
    fx=[0,-1,-1,-1,0,1,1,1]
    fy=[1,1,0,-1,-1,-1,0,1]
    for x in range(1, len(img)-1):
        for y in range(1, len(img[x])-1):
            value = d_lbp(x,y,img)
            val1=0
            for i in range(len(value)):
                if(value[i]==1):
                    val1+=2**(7-i)
            add_dict(dic1, val1)
    return dic1

In [7]:
import cv2
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from scipy.stats import itemfreq
import time
from functools import wraps

df = pd.read_csv(shuffled_labels, header=None)
rows = df.iterrows()

X_addrs = []
X_hist = []
Y_hist = []
#row = rows[0]
j = 0

# uncomment below this for local tetra pattern
dic1 = {}


def create_dic():
    for i in range(len(items)):
        dic1[items[i]] = 0

start_time = time.time()   
'''
for row in rows:
    #print(row[1][1])
    create_dic()
    img = cv2.imread(row[1][0])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    new_x = []
    dic1 = lbp(img,dic1)
    
    x1 = []
    for i in dic1.keys():
        x1.append(dic1[i])
    x1 = np.array(x1)
    #print(x1)
    x1 = x1/np.sum(x1)
    
    new_x = x1.tolist()
    
    X_hist.append(new_x)
    X_addrs.append(row[1][0])
    Y_hist.append(row[1][1])
    if j%100 == 0:
        elapsed_time = time.time() - start_time
        print(str(j) +" "+str(row[1][0])+"  label = "+str(row[1][1]) + " done in " + str(elapsed_time))
        start_time = time.time()
    j = j + 1
    
'''
for row in rows:
    #print(row[1][1])
    im = cv2.imread(row[1][0])
    #print(im.shape)
    im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    radius = 1
    # Number of points to be considered as neighbourers 
    no_points = 8 * (radius)
    lbp = local_binary_pattern(im_gray, no_points, radius, method='nri_uniform')
    # Calculate the histogram
    x = itemfreq(lbp.ravel())
    #print(len(x))
    #feature_length = no_points+2
    feature_length = 59
    dic = {}
    for i in range(len(x)):
        dic[x[i][0]] = x[i][1]
    for i in range(feature_length):
        if i not in dic.keys():
            dic[i] = 0
    x = [[0]*2]*feature_length
    for i in range(feature_length):
        x[i][0] = i
        x[i][1] = dic[i]
    # Normalize the histogram
    #print(len(x))
    if(len(x) != feature_length):
        print(x)
    #x = np.array(x)
    new_x = []
    for i in range(feature_length):
        new_x.append(dic[i])
    new_x = np.array(new_x)
    hist = new_x/np.sum(new_x)
    #hist = x[:, 1]/sum(x[:, 1])
    # Append image path in X_name
    X_addrs.append(row[1][0])
    # Append histogram to X_name
    X_hist.append(hist)
    # Append class label in y_test
    Y_hist.append(row[1][1])
    if(j%100==0):
        elapsed_time = time.time() - start_time
        print(str(j) +" "+str(row[1][0])+"  label = "+str(row[1][1]) + " done in " + str(elapsed_time))
        start_time = time.time()
    j = j + 1


Using TensorFlow backend.


0 dataset/catagory_1\D1_300_3.jpg  label = 1 done in 0.40576815605163574
100 dataset/catagory_3\D3_30_8.jpg  label = 3 done in 4.2475745677948
200 dataset/catagory_0\D112_270_25.jpg  label = 0 done in 4.5064311027526855
300 dataset/catagory_2\D2_90_5.jpg  label = 2 done in 4.33467698097229
400 dataset/catagory_6\D6_0_17.jpg  label = 6 done in 3.9850940704345703
500 dataset/catagory_5\D5_180_5.jpg  label = 5 done in 3.5910046100616455
600 dataset/catagory_1\D1_0_17.jpg  label = 1 done in 3.6768603324890137
700 dataset/catagory_4\D4_0_15.jpg  label = 4 done in 3.456049919128418
800 dataset/catagory_6\D6_0_7.jpg  label = 6 done in 3.0632503032684326
900 dataset/catagory_1\D1_300_6.jpg  label = 1 done in 3.558971881866455
1000 dataset/catagory_0\D112_30_9.jpg  label = 0 done in 3.351088047027588
1100 dataset/catagory_5\D5_270_23.jpg  label = 5 done in 3.3381214141845703
1200 dataset/catagory_2\D2_30_5.jpg  label = 2 done in 3.400059461593628
1300 dataset/catagory_4\D4_270_8.jpg  label = 4 

In [8]:
print(X_addrs[0])
print(X_hist[0])
print(Y_hist[0])
#print(X_hist)
X_hist = np.array(X_hist)
Y_hist = np.array(Y_hist)
print(len(X_addrs))
print(len(X_hist[50]))
print(X_hist.shape)
#print((Y_hist))



dataset/catagory_1\D1_300_3.jpg
[ 0.08319092  0.019104    0.00274658  0.01934814  0.00311279  0.0178833
  0.00183105  0.02130127  0.00354004  0.012146    0.00701904  0.01123047
  0.00720215  0.01000977  0.00628662  0.01019287  0.00738525  0.02026367
  0.00720215  0.0177002   0.00921631  0.01641846  0.00671387  0.01635742
  0.00909424  0.00994873  0.02270508  0.01025391  0.02508545  0.00854492
  0.01928711  0.00732422  0.02380371  0.01538086  0.01641846  0.01434326
  0.01318359  0.01593018  0.01397705  0.01251221  0.01287842  0.01287842
  0.00811768  0.01220703  0.00860596  0.01263428  0.00842285  0.01177979
  0.00872803  0.00384521  0.01971436  0.00634766  0.0144043   0.00445557
  0.01727295  0.00531006  0.01544189  0.08728027  0.15447998]
1
1504
59
(1504, 59)


In [9]:
X_hist = np.array(X_hist)
Y_hist = np.array(Y_hist)

print(X_hist.shape, Y_hist.shape)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(np.array(X_hist), np.array(Y_hist), test_size=0.3)
X_train = np.array(X_train)
print(len(X_train))

'''
for i in range(len(X_train)):
    if(len(X_train[i]) != 26):
       print(len(X_train[i]))
'''


(1504, 59) (1504,)
1052


'\nfor i in range(len(X_train)):\n    if(len(X_train[i]) != 26):\n       print(len(X_train[i]))\n'

In [24]:
#this code is to use the distance function
def feature_distance(feature1, feature2):
    dist = 0.0
    for i in range(len(feature1)):
        dist += abs((feature1[i]*1.0-feature2[i]*1.0)/(1.0+feature1[i]*1.0+feature2[i]*1.0))
        #print(dist)
    return dist
x = X_hist
print(x.shape)
#print(unique_name[X_addrs[0]])
#print(Y_hist[0])
#print(Y_hist[1])
#print(feature_distance(x[0], x[10]))

#60 -> 0.
#50->0.
#40->0.
#35->0.
#30->0.

sm = 0
def clc():
    query_length = 60
    for j in range(3):
        true_val = 0
        false_val = 0
        distance_list = []
        query = x[j]
        query_index = j
        for i in range(len(Y_hist)):
            distance_list.append(feature_distance(query, x[i]))
        unsorted = zip(distance_list, Y_hist)
        sorted_touple = sorted(unsorted, key = lambda element : element[0])
        #print(len(sorted_touple))
        for i in range(query_length):
            if(sorted_touple[i][1] == Y_hist[query_index]):
                true_val = true_val + 1
            else:
                #print("this is wrong "+str(i)+" confusing with "+str(sorted_touple[i][1]))
                false_val = false_val+1
        print(true_val*1.0/query_length*1.0)
clc()

(1504, 59)
0.7333333333333333
0.8166666666666667
0.8166666666666667


In [ ]:
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
#clf = svm.SVC()
#clf = GaussianNB()
#clf = LogisticRegression()
#clf = MLPClassifier(alpha=1)
#clf = AdaBoostClassifier()
clf = RandomForestClassifier()
#clf = DecisionTreeClassifier()

#X_train = np.array(X_train)
print(type(X_train))

clf.fit(X_train,Y_train)

clf.score(X_test, Y_test)

In [ ]:
import pickle
from sklearn.model_selection import cross_val_score
#clf = pickle.load(open('RandomForest_model.sav', 'rb'))
clf = RandomForestClassifier()
clf.fit(X_train,Y_train)
scores = cross_val_score(clf,X_test,Y_test,cv=5)
print((scores))
clf.score(X_test, Y_test)

In [ ]:
import pickle
#for saving model
filename = 'DecisionTree_model.sav'
#pickle.dump(clf, open(filename, 'wb'))
clf = pickle.load(open('DecisionTree_model.sav', 'rb'))
clf.score(X_test, Y_test)
#scores = cross_val_score(clf, X_test, Y_test, cv=20)
#print((scores))

In [ ]:
import numpy as np

X = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
y = np.array([1, 1, 2, 2])

from sklearn.svm import SVC

clf = SVC()
print(type(y))
clf.fit(X, y) 
print(clf.predict([[-0.8, -1]]))

In [ ]:
a = [0]*26
a = np.array(a)
b = a.copy()
b = [0]*14
b = np.array(b)
print(b.shape)
c = a+b
print(c)

In [ ]:
#lbp-> Local derivative ternary lbp last e  tetra 